Note : use Colab python environemnt to run this code , because if you want to use it locally , you might face trouble due to configuration setting of pytorch , make sure all the necessaary files of pretrained model is saved in your specifed directory , all the below listed files are necessary to use pretrained model which are as follow : **Vocab.txt , training_args.bin, tokenizer_config.json, special_tokens, model.safetensors,config.json**

In [5]:
!pip install transformers
!pip install torch
!pip install sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [6]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

Path where Trained model is saved

In [19]:
model_save_path = "/content/drive/MyDrive/ut"

Loading Pretrained model to predict call reason for given transcripts

In [20]:
from transformers import BertForSequenceClassification, BertTokenizer

# Load the model and tokenizer
model = BertForSequenceClassification.from_pretrained(model_save_path)
tokenizer = BertTokenizer.from_pretrained(model_save_path)

In [21]:

labels = ['Voluntary Cancel', 'Booking', 'IRROPS', 'Upgrade', 'Seating',
       'Mileage Plus', 'Checkout', 'Voluntary Change', 'Post Flight',
       'Check In', 'Other Topics', 'Communications', 'Schedule Change',
       'Products and Services', 'Digital Support', 'Disability',
       'Unaccompanied Minor', 'Baggage', 'Traveler Updates', 'ETC']

In [22]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

**Sample Test block to check prediction capabilities of model**

Run next cell toh this text to use the trained model , when you run next cell it ask for the input , input the call transcript , after few seconds you get your call reason for the given transcipt

In [23]:
new_texts = input()  # Example transcript

# Tokenize the new text
new_encodings = tokenizer(new_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")

# Predict
outputs = model(**new_encodings)
predictions = torch.argmax(outputs.logits, dim=-1)

# Convert prediction to label
predicted_reason = label_encoder.inverse_transform(predictions.detach().numpy())

print(predicted_reason)

  Agent: Thank you for calling United Airlines customer service, my name is Steve. How may I help you today?  Customer: Hi, I'm calling to complain about my recent flight. The service was terrible.  Agent: I'm sorry to hear about that. Could you please provide your reservation details so I can pull up your flight information?  Customer: Sure, the flight was from New York to LA last Thursday. Reservation name is John Smith.  Agent: Let me take a look here...okay, I've found your reservation Mr. Smith. What issues did you experience on your flight?  Customer: Well for starters, your airline lost my luggage! I've been waiting over a week and still don't have it. On top of that, the plane was delayed for over 3 hours and we didn't get any explanation or compensation. Then during the flight, the food they served was disgusting and inedible. This was an awful experience from start to finish.   Agent: I apologize for the problems you encountered, that does sound like a very frustrating travel

**Prediction Engine**

Note : for 5157 no of transcripts it takes 2.5 hours to predict all the call reason for all 5157 transcipt , carefull before running the below cells

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/ut2/merged_output.xlsx") # put that files taht contain transcripts

In [ ]:
transcripts = df['call_transcript'].tolist()

In [ ]:
# Assuming the Excel has 'call_id' and 'call_transcript' columns
call_ids = df['call_id'].tolist()
call_transcripts = df['call_transcript'].tolist()


class_labels = ['Voluntary Cancel', 'Booking', 'IRROPS', 'Upgrade', 'Seating',
       'Mileage Plus', 'Checkout', 'Voluntary Change', 'Post Flight',
       'Check In', 'Other Topics', 'Communications', 'Schedule Change',
       'Products and Services', 'Digital Support', 'Disability',
       'Unaccompanied Minor', 'Baggage', 'Traveler Updates', 'ETC']

predicted_reasons = []

# Loop through the transcripts and predict the reason for each
for transcript in call_transcripts:
    # Tokenize the transcript
    new_encodings = tokenizer([transcript], truncation=True, padding=True, max_length=512, return_tensors="pt")

    # Get model predictions
    outputs = model(**new_encodings)
    predictions = torch.argmax(outputs.logits, dim=-1)

    # Map the prediction to the corresponding label
    predicted_reason = class_labels[predictions.item()]  # Get the label from the list
    predicted_reasons.append(predicted_reason)

# Add the predicted reasons to the DataFrame
df['predicted_reason'] = predicted_reasons

